In [1]:
from Frame import Frame
from Camera import Camera
import scipy
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..','fly_model'))
sys.path.insert(0, parent_dir)
import numpy as np
import Plotter
from Skeleton import Skeleton
from Skin import Skin
import plotly.graph_objects as go
import Utils
import pandas as pd
%matplotlib qt

path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'


frames = list(range(900,910,1))

image_name= []
for frame in frames:
    image_name += [f'P{frame}CAM{cam + 1}' for cam in range(4)]
frames = {f'{im_name}.jpg':Frame(path,im_name,idx) for idx,im_name in enumerate(image_name)}


points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:


def get_hull_part(part):
    points_3d_frame = points_3d[part][points_3d[part]['frame'] == 900]
    real_coord_frame = real_coord[real_coord[:,3] == 900,:]
    points_in_ew_frame = np.array([real_coord_frame[points_3d_frame[ax] - 1,idx] for idx,ax in enumerate(['X','Y','Z'])]).T
    points_in_ew_frame_homo = np.hstack((points_in_ew_frame,np.ones([points_in_ew_frame.shape[0],1])))
    points_in_ew_frame  = np.column_stack((points_in_ew_frame,np.arange(1,points_in_ew_frame.shape[0] + 1)))
    return points_in_ew_frame

hull = [get_hull_part(part) for part in list(points_3d.keys())]

In [3]:
frame_number = 900 
frame_names = ['P900CAM1.jpg','P900CAM2.jpg','P900CAM3.jpg','P900CAM4.jpg']
frame_names = [f'P{frame_number}CAM{idx}.jpg' for idx in range(1,5)]

ray_world = np.vstack([frames[frame].camera_center_to_pixel_ray(frames[frame].cm) for frame in  frame_names])
camera_center = np.vstack([frames[frame].X0.T for frame in  frame_names])
point = Utils.triangulate_least_square(camera_center,ray_world)

rot_mat_ew_to_lab = frames['P900CAM1.jpg'].rotation_matrix_from_vectors(frames['P900CAM1.jpg'].R[2,:], [0,0,1])



In [24]:

path_to_mesh = 'G:/My Drive/Research/gaussian_splatting/mesh'
parts = ['body','right_wing','left_wing']
scale = 1/1000
skin_translation = np.array([-0.1-1,0,1])*scale


# initilize skeletone, joints and bones
pitch_body = 0
joints = {'root_no_bone':[1,0,0],
          'neck':[0.6,0,0.3],
          'neck_thorax': [0.6,0,0.3],
          'head':[0.3,0,0],
          'thorax':[-1,0,0],
          'abdomen':[-1.3,0,0] ,
          'right_sp_no_bone': [-0,0,0.3],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,-2.2,0],
          'left_sp_no_bone':[0,0,0.3],
          'left_wing_root':[0,0.3,0],
          'left_wing_tip':[0,2.2,0]
          }

joints = {joint_name: np.array(joint_translation)*scale  for joint_name,joint_translation in joints.items()}

bone_rotation = {'root_no_bone':[0,-pitch_body,0],
          'neck':[0,pitch_body,0],
          'neck_thorax': [0,-25,0],
          'head':[0,0,0],
          'thorax':[0,25,0],
          'abdomen':[0,0,0] ,
          'right_sp_no_bone': [0,pitch_body,0],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,0,0],
          'left_sp_no_bone':[0,pitch_body,0],
          'left_wing_root':[-0,0,0],
          'left_wing_tip':[0,0,0]
          }

parent_child = {'root_no_bone':['neck','neck_thorax','right_sp_no_bone','left_sp_no_bone'],
                      'neck':['head'],
                      'neck_thorax':['thorax'],
                      'thorax':['abdomen'],
                      'right_sp_no_bone':['right_wing_root'],
                      'right_wing_root': ['right_wing_tip'],
                      'left_sp_no_bone': ['left_wing_root'],
                      'left_wing_root':['left_wing_tip']}


skin = Skin(parts,path_to_mesh,scale = 1)
skin.translate_ptcloud_skin(skin_translation)
skeleton = Skeleton(parent_child,joints,bone_rotation,skin)




In [25]:

# initilize skeletone, joints and bones
pitch_body = 40
new_translation =  {'root_no_bone':[1,0,0],
          'neck':[0.6,0,0.3],
          'neck_thorax': [0.6,0,0.3],
          'head':[0.3,0,0],
          'thorax':[-1,0,0],
          'abdomen':[-1.3,0,0] ,
          'right_sp_no_bone': [-0,0,0.3],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,-2.2,0],
          'left_sp_no_bone':[0,0,0.3],
          'left_wing_root':[0,0.3,0],
          'left_wing_tip':[0,2.2,0]
          }
new_translation = {joint_name: np.array(joint_translation)*scale  for joint_name,joint_translation in new_translation.items()}

new_rotation = {'root_no_bone':[0,-pitch_body,0],
          'neck':[0,pitch_body,0],
          'neck_thorax': [0,-25,0],
          'head':[0,0,0],
          'thorax':[0,25,0],
          'abdomen':[0,0,0] ,
          'right_sp_no_bone': [0,pitch_body,0],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,0,0],
          'left_sp_no_bone':[0,pitch_body,0],
          'left_wing_root':[-0,0,0],
          'left_wing_tip':[0,0,0]
          }
ls_rotated = np.dot(rot_mat_ew_to_lab,point[:,np.newaxis]).T

new_translation['root_no_bone'] = ls_rotated[0] + new_translation['root_no_bone']
skeleton.update_skeleton(new_rotation,new_translation)
transformed_points, transformed_normals = skin.rotate_skin(skeleton)





In [26]:
import matplotlib.pyplot as plt
transformed_points_to_rotate = transformed_points - np.mean(transformed_points,axis = 0)
transformed_points_ew = np.dot(rot_mat_ew_to_lab.T,transformed_points.T).T
transformed_points_ew = transformed_points_ew 
frame_name = 'P900CAM4.jpg'
transformed_points_homo = frames[frame_name].homogenize_coordinate(transformed_points_ew)
projeced = frames[frame_name].project_on_image(transformed_points_homo)

# plt.imshow(frames[frame_name].image_no_bg)
# plt.scatter(projeced[:,0],projeced[:,1])

In [27]:


camera_center_rotated = np.dot(rot_mat_ew_to_lab,camera_center.T).T
ray_world_rotated = np.dot(rot_mat_ew_to_lab,ray_world.T).T
points_in_ew_frame_rotated = [np.dot(rot_mat_ew_to_lab,part[:,0:3].T).T for part in hull]


vector_dir = np.array(ray_world_rotated - camera_center_rotated )[:,0:3]
vector_dir_norm= (vector_dir.T/np.linalg.norm(vector_dir,axis = 1)).T



# new_point = np.dot(vector_dir[0][0:3],np.array([0,0,1])[:,np.newaxis]).T[0]
# scales = np.dot(vector_dir[0][0:3],np.array([0,0,1]))
ray_world2  = camera_center_rotated + vector_dir_norm*np.array([[0.3,0.22,0.22,0.22]]).T
camera_center2  = camera_center_rotated + vector_dir_norm*np.array([[0.26,0.16,0.16,0.16]]).T


In [28]:
fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})
fig.show()

In [132]:
fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})

marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
# fig = go.Figure()

marker_dict = {'size': 5, 'color': [[255,0,0],[0,255,0]]}


Plotter.scatter3d(fig,points_in_ew_frame_rotated[0],'hull_body', mode = 'markers',marker_dict =  {'color': 'blue'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[1],'hull_rwing', mode = 'markers',marker_dict = { 'color': 'magenta'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[2],'hull_lwing', mode = 'markers',marker_dict = {'color':'cyan'}) 

marker_dict_point = {'size': 5, 'color': 'magenta'}

Plotter.scatter3d(fig,ls_rotated,'least square', mode = 'markers',marker_dict = marker_dict_point) 

fig.show()

In [54]:
fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})

marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
# fig = go.Figure()

marker_dict = {'size': 5, 'color': [[255,0,0],[0,255,0]]}

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]

Plotter.scatter3d(fig,points_in_ew_frame_rotated[0],'hull_body', mode = 'markers',marker_dict =  {'color': 'blue'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[1],'hull_rwing', mode = 'markers',marker_dict = { 'color': 'magenta'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[2],'hull_lwing', mode = 'markers',marker_dict = {'color':'cyan'}) 

marker_dict_point = {'size': 5, 'color': 'magenta'}

Plotter.scatter3d(fig,ls_rotated,'least square', mode = 'markers',marker_dict = marker_dict_point) 

fig.show()

In [32]:

fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})
fig.show()

In [34]:
ray_world2

array([[-0.00123098, -0.0229936 , -0.00810553],
       [ 0.00453605, -0.01595115,  0.00059876],
       [-0.009139  , -0.01799263,  0.00017689],
       [-0.00048583, -0.01106921, -0.01935558]])

In [129]:
fig = go.Figure()

marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
# fig = go.Figure()

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center, ray_world,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center, ray_world,range(4))]


marker_dict = {'size': 5, 'color': 'blue'}

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]

fig.show()


In [182]:

transformed_points_to_rotate = transformed_points - np.mean(transformed_points,axis = 0)
transformed_points_ew = np.dot(rot_mat.T,transformed_points_to_rotate.T).T
transformed_points_ew = transformed_points_ew + np.mean(transformed_points,axis = 0)

transformed_points_homo = frames['P900CAM4.jpg'].homogenize_coordinate(transformed_points_ew)
projeced = frames['P900CAM4.jpg'].project_on_image(transformed_points_homo)

plt.imshow(frames['P900CAM4.jpg'].image_no_bg)
plt.scatter(projeced[:,1],projeced[:,0])

In [183]:

fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points_ew,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})

marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
# fig = go.Figure()

marker_dict = {'size': 5, 'color': [[255,0,0],[0,255,0]]}

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]


Plotter.scatter3d(fig,point[:,np.newaxis].T,'least square', mode = 'markers',marker_dict = marker_dict_point) 

fig.show()